In [1]:
from datetime import datetime
import statistics

import gurobipy as gp
from gurobipy import GRB

In [3]:
# Read the problem
name="00001"
file = open( "J10/"+name+".txt", 'r')

line_no = 0;
c=0
problem = {"items": []}  # a dictionary with various propertie + items a list of dicts.
problem["name"]=name
while True:
    line = file.readline()
    if not line:
        break
    line = line.replace("\n", "")
    if line_no==0:
        content = line.split(" ")
        problem["n"]=int(content[0])
    elif line_no in range(problem["n"]+4,problem["n"]+8):
        content = line.split(" ")
        problem[content[1][1:]]=int(content[0])
    elif line_no in range(3,problem["n"]+3):
        content = line.split(" ")
        item = {"id_x": int(content[0]), "p": int(content[1]), "w": int(content[2]),"x": int(content[3])}
        problem["items"].append(item)
    else:
        pass # do nothing, just comment lines
    line_no += 1
file.close()
problem["sorted_items"] = sorted(problem["items"], key=lambda x: (-x['p'] / x['w'], -x['w']))


In [4]:
profits = list(problem["items"][i]["p"] for i in range(int(problem["n"])))
weights = list(problem["items"][i]["w"] for i in range(int(problem["n"])))
n=int(problem["n"])
capacity = problem["c"]

In [8]:
#Model
start_date = datetime.now()
m = gp.Model()

# add decision variables
x: None = m.addVars(n, vtype=GRB.BINARY, name='x')

# set objective function
m.setObjective(gp.quicksum(profits[i] * x[i] for i in range(n)), GRB.MAXIMIZE)

# add constraint
constraint = m.addConstr((gp.quicksum(weights[i] * x[i] for i in range(n)) <= capacity), name="Capacity")

# Add Lazy Constraints
#m.setParam(GRB.Param.LogToConsole, 0) # 1 for verbose log
#m.setParam('TimeLimit', timeOut )
# m.setParam('Method', 5)  # set  2 the method to branch-and-price
# m.setParam('MIPFocus', 1)  # set to 1 to focus on finding feasible solutions quickly
#m.update()
# print(f"{m.getRow(constraint)} {constraint.Sense} {constraint.RHS}")

# Set the MIP gap to 10% (0.1)
m.setParam('MIPGap', 0.000001)
m.optimize()  # solve the model
end_date = datetime.now()



Set parameter MIPGap to value 1e-06
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 400 columns and 400 nonzeros
Model fingerprint: 0x04e8ccc2
Variable types: 0 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+05]
  Objective range  [2e+00, 5e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+06, 1e+06]
Found heuristic solution: objective 1003069.0000
Presolve removed 0 rows and 37 columns
Presolve time: 0.00s
Presolved: 1 rows, 363 columns, 363 nonzeros
Variable types: 0 continuous, 363 integer (362 binary)
Found heuristic solution: objective 1003160.0000

Root relaxation: objective 1.004001e+06, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

   

In [ ]:
# objective: 1003988